## Find chats that are annotated multiple times

In [72]:
import pandas as pd
import numpy as np
from glob import glob

In [73]:
# # load file
file_paths = glob(r"C:\Users\Michael's  Laptop\Documents\XSELL\Verizon Data Set 9\*.csv")
df = pd.concat([pd.read_csv(file_path,error_bad_lines=False,sep = ';') for file_path in file_paths])
# for filepath in file_paths:
# filepath = file_paths[0]
#     #     print filepath
# df = pd.read_csv(file_path, sep='\t',error_bad_lines=False)

In [74]:
df.columns

Index([u'engagement_id', u'agent_id', u'start_url', u'topic', u'node_name',
       u'start_time', u'message_sender', u'message_text', u'clicked',
       u'nexted', u'recommendations', u'converted', u'agent_group',
       u'agent_location'],
      dtype='object')

In [75]:
print('Total Chats: ' + str(df.engagement_id.nunique()))

Total Chats: 10042


In [76]:
df.Topic.unique()

AttributeError: 'DataFrame' object has no attribute 'Topic'

In [27]:
# creates a dataframe with a list of all classifications given for each engagement_id
duplicates = df[~df.Topic.isnull()].groupby('engagement_id')['Topic'].apply(list).to_frame().reset_index()

In [28]:
# how many times an engagement was annotated
duplicates['topic_count'] = duplicates.Topic.apply(lambda x: len(x))
# how many times an engagment was annotated with a unique value (e.g. ['Add', 'Add'])
duplicates['unique_topic_count'] = duplicates.Topic.apply(lambda x: len(set(x)))

In [29]:
# gets engagement_ids for single annotation chats
clean_ids = duplicates[(duplicates.topic_count == 1)].engagement_id.tolist()
# gets engagement_ids for multiple annotation chat with same annotation
murky_ids = duplicates[(duplicates.topic_count > 1) & (duplicates.unique_topic_count == 1)].engagement_id.tolist()
# gets engagement_ids for multiple annotation chats
dirty_ids = duplicates[(duplicates.topic_count > 1) & (duplicates.unique_topic_count > 1)].engagement_id.tolist()

In [30]:
len(clean_ids)

2851

In [31]:
len(murky_ids)

130

In [32]:
len(dirty_ids)

41

In [33]:
# to files
file_path = r"C:\Users\Michael's  Laptop\Documents\XSELL\Verizon Data Set 6 (Clean ID's)\cleanID.csv"
df[df.engagement_id.isin(clean_ids)].to_csv(file_path.replace('.csv','_clean.csv'))
# df[df.engagement_id.isin(murky_ids)].to_csv(file_path.replace('.csv','_murky.csv'))
# df[df.engagement_id.isin(dirty_ids)].to_csv(file_path.replace('.csv','_dirty.csv'))

# Lenckos Approach

In [77]:
def remove_dupes(df, exclude_ids=None):
    
    unchecked = set(df.engagement_id.unique()) 
    if exclude_ids:
        unchecked = unchecked - set(exclude_ids)
    print(len(unchecked))
    current_id = ''
    rows = []
    for row in df.itertuples():
        if row.engagement_id != current_id:
            # we should be done with current_id
            if current_id in unchecked:
                unchecked.remove(current_id)
            # check if this engagement id hasn't been seen yet
            if row.engagement_id in unchecked:
                current_id = row.engagement_id
            else:
                current_id = ''
        if row.engagement_id == current_id:
            rows.append(row)

    print(len(unchecked))
    return pd.DataFrame(rows)

In [81]:
df2 = remove_dupes(df)
remove_dupes(df2)

10042
1
10042
1


,Index,_1,engagement_id,agent_id,start_url,topic,node_name,start_time,message_sender,message_text,clicked,nexted,recommendations,converted,agent_group,agent_location
0,0,0,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:00:51,visitor_message,Connect to Live Chat,0,0,NaN,f,WirelessSales,NACOGDOCHES
1,1,1,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:03:12,agent_message,"Welcome to Verizon Wireless Sales Chat, I'll b...",0,0,NaN,f,WirelessSales,NACOGDOCHES
2,2,2,"""-5787867841934331022""",3466689398@verizon,NaN,add,add_standard_acknowledge,2019-05-16 00:03:31,visitor_message,"good, I want to know if I am eligible for a ne...",0,0,It would be my pleasure to help you with addin...,f,WirelessSales,NACOGDOCHES
3,3,3,"""-5787867841934331022""",3466689398@verizon,NaN,add,add_standard_acknowledge,2019-05-16 00:04:09,agent_message,Awesome! I would be more than happy to help y...,0,0,It would be my pleasure to help you with addin...,f,WirelessSales,NACOGDOCHES
4,4,4,"""-5787867841934331022""",3466689398@verizon,NaN,add,add_standard_acknowledge,2019-05-16 00:04:36,visitor_message,thanks,0,0,It would be my pleasure to help you with addin...,f,WirelessSales,NACOGDOCHES
5,5,5,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:05:00,agent_message,Sure thing!,0,0,NaN,f,WirelessSales,NACOGDOCHES
6,6,6,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:05:33,agent_message,I've pulled up your credit application and wil...,0,0,NaN,f,WirelessSales,NACOGDOCHES
7,7,7,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:05:36,agent_message,"Please <a href=""https://inspicio.verizonwirele...",0,0,NaN,f,WirelessSales,NACOGDOCHES
8,8,8,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:05:54,visitor_message,thank one moment please,0,0,NaN,f,WirelessSales,NACOGDOCHES
9,9,9,"""-5787867841934331022""",3466689398@verizon,NaN,add,NaN,2019-05-16 00:06:11,agent_message,No problem!,0,0,NaN,f,WirelessSales,NACOGDOCHES


NameError: name 'exclude_ids' is not defined

In [68]:
directory = r"C:\Users\Michael's  Laptop\Documents\XSELL\Verizon Data Set 9/"
filename = (directory + 'dataset9combined.csv')
df2.to_csv(filename, index=False)

In [49]:
df.head(15)

,#,% Done,Notes,Notes/Promos,SubTopic,Subtopic,Topic,Unnamed: 19,Unnamed: 20,agent_group,...,converted,engagement_id,message_text,nexted,node_name,recommendations,role,start_time,start_url,topic
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",.,0.0,NaN,NaN,A,2019-05-15 16:02:30,https://www.verizonwireless.com/plans/,upgrade
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",166923,0.0,NaN,NaN,C,2019-05-15 16:02:32,https://www.verizonwireless.com/plans/,upgrade
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",Thank you,0.0,NaN,NaN,A,2019-05-15 16:02:44,https://www.verizonwireless.com/plans/,upgrade
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",Did you need a new number for that new line?,0.0,NaN,NaN,A,2019-05-15 16:03:34,https://www.verizonwireless.com/plans/,upgrade
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",yes,0.0,NaN,NaN,C,2019-05-15 16:03:38,https://www.verizonwireless.com/plans/,upgrade
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",i need a new number!,0.0,NaN,NaN,C,2019-05-15 16:03:46,https://www.verizonwireless.com/plans/,upgrade
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""","Ok, and what device were you thinking of?",0.0,NaN,NaN,A,2019-05-15 16:03:55,https://www.verizonwireless.com/plans/,upgrade
7,NaN,NaN,Galaxy Note 9,NaN,NaN,Samsung Tablet,Add,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",i want to add a galaxy Note 9.,0.0,NaN,NaN,C,2019-05-15 16:04:07,https://www.verizonwireless.com/plans/,upgrade
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""","Awesome, we offer that device in 2 storage siz...",0.0,NaN,NaN,A,2019-05-15 16:04:51,https://www.verizonwireless.com/plans/,upgrade
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WirelessSales,...,f,"""724900167288993567""",128gb,0.0,NaN,NaN,C,2019-05-15 16:04:56,https://www.verizonwireless.com/plans/,upgrade


## Remove duplicate chats

In [14]:
df = df[df.engagement_id.isin(murky_ids)]

In [15]:
df = df.reset_index()
data = df.merge(df.engagement_id.drop_duplicates(keep='first').to_frame().reset_index(), how='left', on='engagement_id')

In [16]:
data['index_delta'] = data.index - data.index_y

In [17]:
cap = 0
cap2 = 0
for i in range(200):
    dups = data[(data.index_delta < i) & (~data.Topic.isnull())].groupby('engagement_id')['Topic'].apply(list).to_frame().reset_index()
    dups['topic_count'] = dups.Topic.apply(lambda x: len(x))
    dups['unique_topic_count'] = dups.Topic.apply(lambda x: len(set(x)))
    c = dups[(dups.topic_count == 1) & (dups.unique_topic_count == 1)].engagement_id.nunique()
    d = dups[(dups.topic_count > 1) & (dups.unique_topic_count == 1)].engagement_id.nunique()
    if d > cap or c > cap2:
        print(i, d, c)
        cap = d
        cap2 = c

(1, 0, 148)
(2, 0, 168)
(3, 0, 178)
(4, 0, 192)
(5, 0, 209)
(6, 0, 219)
(7, 1, 227)
(8, 1, 234)
(9, 1, 237)
(10, 1, 240)
(11, 1, 246)
(12, 1, 250)
(14, 1, 256)
(15, 1, 257)
(16, 1, 258)
(17, 1, 263)
(18, 1, 267)
(19, 1, 272)
(20, 1, 279)
(21, 1, 282)
(22, 1, 285)
(23, 1, 289)
(24, 1, 292)
(25, 1, 294)
(26, 1, 299)
(27, 1, 302)
(28, 1, 303)
(29, 1, 304)
(30, 2, 303)
(34, 2, 304)
(35, 2, 305)
(36, 2, 306)
(37, 2, 307)
(38, 2, 308)
(39, 2, 309)
(43, 2, 311)
(44, 2, 312)
(48, 2, 314)
(54, 2, 315)
(55, 2, 316)
(67, 2, 317)
(68, 2, 318)
(102, 2, 319)
(126, 2, 320)


In [18]:
# take max i with max c and min d
i = 29
data[(data.index_delta < i)][['message_text','role','engagement_id','Topic']].to_csv(file_path.replace('.csv','_murky_scrubbed.csv'))

# Combine unique chats

In [40]:
df = pd.concat([pd.read_csv(file_path.replace('.csv','_murky_scrubbed.csv')), pd.read_csv(file_path.replace('.csv','_clean.csv'))])

In [41]:
df.engagement_id.nunique()

11001

In [42]:
df[['message_text','role','engagement_id','Topic']].to_csv(file_path.replace('.csv','_unique.csv'), index=False)

## Orest Approach

In [48]:
df = df[df.engagement_id.isin(murky_ids)]

In [68]:
i = 0
for id in murky_ids:
    indexes = df[df.engagement_id == id].index
    half = len(indexes)//2
    a = df.loc[indexes[:half]].message_text.tolist()
    b = df.loc[indexes[half:]].message_text.tolist()
#     print(a)
#     print(b)
    if (a == b):
        i += 1
print i, len(murky_ids)

4 321
